In [1]:
import numpy as np
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
import os
import sys
import pickle
from struct import *
import pandas as pd
import seaborn as sns
import mixture
sns.set_style("white")
import warnings
warnings.filterwarnings("ignore")
import collections
from math import pi, cos, sin, cosh, tanh
from scipy.spatial.transform import Rotation as Rot
import cv2
import plotting
from plotting import *
import interpolation
from interpolation import *
import ficks
from ficks import *
import structure
from structure import *
import scipy.interpolate
import scipy.integrate
import scipy.stats
from sklearn.linear_model import RANSACRegressor, TheilSenRegressor, LinearRegression
from sklearn.mixture import GaussianMixture
from scipy.stats import norm
from scipy.signal import argrelextrema
from statistics import median

In [2]:
#
# DO NOT TOUCH. Define colors and states for plotting events.
#
STATES = ['fix', 'sac', 'smp', 'vor', 'blink', 'fix_blink', 'sac_blink', 'other', 'loss']
COLORS = {'fix':'red', 'sac':'lime', 'smp':'purple', 'vor':'brown', 'blink':'dodgerblue', 'fix_blink':'darkorchid', 'sac_blink':'turquoise', 'other':'grey', 'loss':'gold'}
BORDERS = {'fix':'magenta', 'sac':'green', 'smp':'blue', 'vor':'tan', 'blink':'royalblue', 'fix_blink':'darkviolet', 'sac_blink':'lightseagreen', 'other':'black', 'loss':'yellow'}
BINS = 50

# All participants and their respective vision loss
vision_loss = {'P2':'CVL', 'P3':'CVL', 'P4':'Other', 'P5':'PVL', 'P6':'CVL', 'P8':'CVL', 'P9':'CVL', 'P10':'CVL',
                'P11':'PVL', 'P12':'PVL', 'P13':'CVL', 'P14':'PVL', 'P15':'PVL', 'P16':'PVL', 'P17':'PVL',
                'P18':'CVL', 'P19':'CVL', 'P21':'CVL', 'P22':'PVL', 'P23':'PVL', 'P24':'PVL', 'P25':'CVL'}


### probability functions

In [3]:
def gaussian(x, mu, std):
    return (1/(std*np.sqrt(2*np.pi)))*np.exp(-0.5*((x-mu)/std)**2)

### functions used for Carpenter's theorem

In [4]:
def carpenter(a):
    """
    A function that takes in amplitude (deg) and returns duration (sec).
    """
    return (21 + 2.2*a)/1000

In [5]:
# in_path = '/data/Isabella/thesis_spring2022/event_detect_in/'

# for fname in os.listdir(in_path):
#     ID = fname.partition('.csv')[0] 
#     if 'hand_wash' in ID:
#         DEVICE, _, __, eye, subID = ID.split('_')
#         task = 'handwash'
#         os.rename(in_path+fname, in_path+f'{DEVICE}_{task}_{eye}_{subID}')

## Steps to event detection:

### Step 0: Pre-process
- 0.1 Calculate intersample velocity v.
- 0.2 Remove noise by replacing x,y,v with NAN where v > 1000 and v<=0.
- 0.3 Label loss where there is a difference in time greater than 400 ms between samples.
- 0.4 Up-sample angular positional features x, y where there are gaps in data above 3xSAMPLE_RATE but less than 400 ms using cubic intervariate spline method.
- 0.5 Interpolate the upsampled angular positional features (Dx, Dy) and find their derivatives.

### Step 1: Low- vs High-speed events
- Calculate feature Dn = sqrt(diff(dDx/dt)+diff(dDy/dt))
- Run EM-GMM to calculate priors and likelihoods for low- and high-speed.
- Calculate probabilities for low- and high-speed sample-by-sample.
- Label samples where p(L) >= p(H) as low-speed, otherwise high-speed. 
- Create events_df from labelled events.

### Step 2: Within movement classify saccades and blinks
- Calculate feature Dv = abs(max-min) from diff(dDy/dt) (vertical interpolated velocity) for all non-loss events.
- Run EM-GMM on distribution of Dv in high-speed events to calculate priors and likelihoods for sac or blink.
- Calculate joint probabilities for sac and blink event-by-event, conditional on the probability of the event being a high-speed event.

### Step 3: Within fixations distinguish between fixations and smooth pursuits
(which could also be VOR since we don't account for head rotation)
- Calculate amplitude feature as Da = sqrt((Dx_t-Dx_0)^2+(Dy_t-Dy_0)^2) for all non-loss events (unless fixation, then Da = std(x)+std(y).
- Run EM-GMM on distribution of Da in low-speed events to calculate priors and likelihoods for fix or smp.
- Calculate probabilities for fix or smp event-by-event, conditional on the probability of the event being a low-speed event.

### Step 4: Classify events
- For each event in events_df, classify the event as that which has the highest probability

## Run event detection, one dataset at a time

In [6]:
varjo_to_use = {'P2':['cereal'], 'P3':['cereal', 'sandwich'], 'P5':['cereal', 'sandwich'], 
                'P6':['cereal', 'sandwich'], 'P8':['sandwich'], 'P9':['cereal', 'sandwich'],
                'P13':['cereal', 'sandwich'], 'P14':['cereal', 'sandwich'], 'P15':['cereal'],
                'P18':['cereal', 'sandwich'], 'P19':['cereal', 'sandwich'],
                'P21':['cereal'], 'P22':['cereal', 'sandwich'], 'P23':['cereal', 'sandwich'],
                'P24':['cereal', 'sandwich'], 'P25':['cereal', 'sandwich']}

In [7]:
sample_rates = {'varjo':200, 'PI':66}
in_path = '/data/Isabella/thesis_spring2022/event_detect_in/'
out_path = '/data/Isabella/thesis_spring2022/event_detect_out_final/'

NUM = 1
for fname in os.listdir(in_path):
    
    ID = fname.partition('.csv')[0]
    DEVICE, task, eye, subID = ID.split('_')
    SAMPLE_RATE = sample_rates[DEVICE]
    MIN_FIX_DUR = 0.1
    
    # only run on varjo
    if DEVICE == 'PI':
        continue
    
    # do only varjo used in NN
    if subID in varjo_to_use.keys():
        if task not in varjo_to_use[subID]:
            continue
    else: continue
        
    # only run on datasets that have not been run
    if fname in os.listdir(out_path):
        print(f'[{NUM}] {ID} already exists in {out_path}')
        NUM += 1
        continue
    
#     # Initialize output dataframe for each combination of HMD:task:eye:subID
#     sequences = pd.DataFrame({})
#     test_seq = sequences.copy()
#     test_seq_51 = sequences.copy()
#     test_seq_52 = sequences.copy()
    
    print(f'{ID} processing...')

    df = pd.read_csv(in_path+fname)
    df['event'] = 'other'
    
    # Detect loss and replace with NANs
    max_interp_dur = 0.4 # seconds
    to_drop = []
    for idx, row in df[1:].copy().iterrows():
        if idx in to_drop:
            continue
        i = idx+1
        prev_row = df.loc[idx-1]
        dt = row.time - prev_row.time
        if dt > max_interp_dur:
            df.loc[idx-1:i, 'event'] = 'loss'
            df.loc[idx-1:i, 'x_deg':'issy'] = np.NAN

    df['x_deg'] = df['x_deg'].astype(np.float)
    df['y_deg'] = df['y_deg'].astype(np.float)
    
    # Calculate interpolated velocity
    interp_v = interpolate(df, SAMPLE_RATE, feat='iss')
    df['Dn'] = interp_v(df.time)
    
    #
    # Step 1. Low-vs-High-speed events
    #
    # set features
    Dn = 'Dn' # interpolated iss (quadratic)

    # estimate model parameters with the EM algorithm
    Dn_mixture = GaussianMixture(n_components=2).fit(df.loc[df.event!='loss', Dn].to_numpy().reshape(-1,1))
    Dn_means_hat = Dn_mixture.means_.flatten()
    Dn_weights_hat = Dn_mixture.weights_.flatten()
    Dn_sds_hat = np.sqrt(Dn_mixture.covariances_).flatten()

    # fix should have smaller mean than non-fix
    fixi = np.argmin(Dn_means_hat)
    nfixi = np.argmax(Dn_means_hat)
    Dn_mu1, Dn_sd1 = Dn_means_hat[fixi], Dn_sds_hat[fixi]
    Dn_mu2, Dn_sd2 = Dn_means_hat[nfixi], Dn_sds_hat[nfixi]

    prior_fix = Dn_weights_hat[fixi]
    prior_nonfix = Dn_weights_hat[nfixi]
    
    # sklearn's gaussian mixture model uses EM algorithm to estimate model parameters
    df['P_fix'] = 0.0
    df['P_nonfix'] = 0.0
    df['L_fix'] = 0.0
    df['L_nonfix'] = 0.0
    for idx, row in df.iterrows():
        if row[Dn] < Dn_mu1:
            L_fix = norm.pdf(Dn_mu1, Dn_mu1, Dn_sd1)
        else:
            L_fix = norm.pdf(row[Dn], Dn_mu1, Dn_sd1)
        if row[Dn] > Dn_mu2:
            L_nonfix = norm.pdf(Dn_mu2, Dn_mu2, Dn_sd2)
        else:
            L_nonfix = norm.pdf(row[Dn], Dn_mu2, Dn_sd2)
        df.loc[idx, 'L_fix'] = L_fix
        df.loc[idx, 'L_nonfix'] = L_nonfix
        df.loc[idx, 'P_fix'] = (L_fix*prior_fix)/(L_fix*prior_fix + L_nonfix*prior_nonfix)
        df.loc[idx, 'P_nonfix'] = (L_nonfix*prior_nonfix)/(L_fix*prior_fix + L_nonfix*prior_nonfix)
    
    # Classify fixations as those with probability greater than or equal to 50%.
    df['event'] = np.where(df.event!='loss', np.where(df.P_fix>=0.5, 'fix', df.event), df.event)

    # Make events_df (need to iterate twice)
    # Ignore fixation events where the duration is less than the minimum duration of a fixation.
    events_df = make_events_df(df, interp_v, SAMPLE_RATE, MIN_FIX_DUR)
    # running it again means that consecutive events are merged into one
    df = map_to_stream(events_df, df)
    events_df = make_events_df(df, interp_v, SAMPLE_RATE, MIN_FIX_DUR)
    
    # mark events with calculus error above 1.5 as noise (non-movements)
    events_df.loc[events_df.calculus_error>1.5, 'event'] = 'noise'
    df = map_to_stream(events_df, df)
    

    #
    # Step 2: Sac vs Blink
    #
    # set Dv feature
    Dv = 'calculus_error'

    # estimate model parameters with the EM algorithm for non-fix events
    #  if an error is found here, then that is because the model does not have enough high-speed events
    #  and has instead found a lot of noise in the dataset so it should not be processes
    #  (it means consecutive non-nan events are not above Nyquist threshold)
    try:
        Dv_mixture = GaussianMixture(n_components=2).fit(events_df.loc[events_df.event=='other', Dv].to_numpy().reshape(-1,1))
    except:
        print(f'{ID} FAILED - noisy data below Nyquist threshold')
        continue
    Dv_means_hat = Dv_mixture.means_.flatten()
    Dv_weights_hat = Dv_mixture.weights_.flatten()
    Dv_sds_hat = np.sqrt(Dv_mixture.covariances_).flatten()

    # sac should have smaller mean than blink
    saci = np.argmin(Dv_means_hat)
    blinki = np.argmax(Dv_means_hat)
    Dv_mu1, Dv_sd1 = Dv_means_hat[saci], Dv_sds_hat[saci]
    Dv_mu2, Dv_sd2 = Dv_means_hat[blinki], Dv_sds_hat[blinki]

    prior_sac = Dv_weights_hat[saci]
    prior_blink = Dv_weights_hat[blinki]
    
    # sklearn's gaussian mixture model uses EM algorithm to estimate model parameters
    events_df['P_sac'] = 0.0
    events_df['P_blink'] = 0.0
    events_df['L_sac'] = 0.0
    events_df['L_blink'] = 0.0
    for idx, row in events_df.loc[events_df.event!='noise'].iterrows():
        if row[Dv] <= Dv_mu1:
            L_sac = norm.pdf(Dv_mu1, Dv_mu1, Dv_sd1)
        else:
            L_sac = norm.pdf(row[Dv], Dv_mu1, Dv_sd1)
        if row[Dv] > Dv_mu2:
            L_blink = norm.pdf(Dv_mu2, Dv_mu2, Dv_sd2)
        else:
            L_blink = norm.pdf(row[Dv], Dv_mu2, Dv_sd2)
        events_df.loc[idx, 'L_sac'] = L_sac
        events_df.loc[idx, 'L_blink'] = L_blink
        events_df.loc[idx, 'P_sac'] = ((L_sac*prior_sac)/(L_sac*prior_sac + L_blink*prior_blink))*row.P_nonfix
        events_df.loc[idx, 'P_blink'] = ((L_blink*prior_blink)/(L_sac*prior_sac + L_blink*prior_blink))*row.P_nonfix
    
    
    #
    # Step 3: Fix vs Smp
    #
    # set Da feature
    Da = 'dispersion'

    # estimate model parameters with the EM algorithm
    Da_mixture = GaussianMixture(n_components=2).fit(events_df.loc[events_df.event=='fix', Da].to_numpy().reshape(-1,1))
    Da_means_hat = Da_mixture.means_.flatten()
    Da_weights_hat = Da_mixture.weights_.flatten()
    Da_sds_hat = np.sqrt(Da_mixture.covariances_).flatten()

    # fix should have smaller mean than smp/vor
    fixi = np.argmin(Da_means_hat)
    smpi = np.argmax(Da_means_hat)
    Da_mu1, Da_sd1 = Da_means_hat[fixi], Da_sds_hat[fixi]
    Da_mu2, Da_sd2 = Da_means_hat[smpi], Da_sds_hat[smpi]

    prior_ff = Da_weights_hat[fixi]
    prior_smp = Da_weights_hat[smpi]
    
    # sklearn's gaussian mixture model uses EM algorithm to estimate model parameters
    events_df['P_ff'] = 0.0
    events_df['P_smp'] = 0.0
    events_df['L_ff'] = 0.0
    events_df['L_smp'] = 0.0
    for idx, row in events_df.iterrows():
        if row[Da] <= Da_mu1:
            L_ff = norm.pdf(Da_mu1, Da_mu1, Da_sd1)
        else:
            L_ff = norm.pdf(row[Da], Da_mu1, Da_sd1)
        if row[Da] > Da_mu2:
            L_smp = norm.pdf(Da_mu2, Da_mu2, Da_sd2)
        else:
            L_smp = norm.pdf(row[Da], Da_mu2, Da_sd2)
        events_df.loc[idx, 'L_ff'] = L_ff
        events_df.loc[idx, 'L_smp'] = L_smp
        events_df.loc[idx, 'P_ff'] = (L_ff*prior_ff)/(L_ff*prior_ff + L_smp*prior_smp)*row.P_fix
        events_df.loc[idx, 'P_smp'] = (L_smp*prior_smp)/(L_ff*prior_ff + L_smp*prior_smp)*row.P_fix
    
    
    #
    # Step 4: Classify events and evaluate error metrics
    #
    # Classify events as that with the largest probability
    s = ['fix', 'smp', 'sac', 'blink']
    for idx, row in events_df[(events_df.event!='loss')&(events_df.event!='noise')].iterrows():
        i = np.argmax([row.P_ff, row.P_smp, row.P_sac, row.P_blink])
        events_df.loc[idx, 'event'] = s[i]
        
    # map the new events_df to df
    df = map_to_stream(events_df, df)
    
    
    #
    # Step 4.1: Adjustments
    #
    # Merge events broken up by noise with recalculated features and drop the noise
    to_drop = []
    for idx, row in events_df.copy().iterrows():
        if idx == 0 or idx == len(events_df)-1:
            continue

        prev_row = events_df.loc[idx-1]
        next_row = events_df.loc[idx+1]

        # if noise occurs between the same event
        if row.event == 'noise' and prev_row.event == next_row.event:

            events_df = merge_events(df, events_df, idx-1, idx+1, prev_row.event, interp_v, SAMPLE_RATE)
            to_drop.extend([idx,idx+1])

    # drop events that are now merged
    events_df = events_df.drop(to_drop)
    events_df.reset_index(inplace=True)
    del events_df['index']

    # map the updated events_df to df
    df = map_to_stream(events_df, df)

    # adjust amplitude and related features for non-fixations between two fixations
    events_df = adjust_amp_dur(events_df, df, interp_v)
    

    # ADJUSTMENTS
    # relabel smp as sac if carpenter error is less than threshold
    avg_sac_carp_error_plus_two_std = events_df[events_df.event=='sac'].carpenter_error.mean()+2*events_df[events_df.event=='sac'].carpenter_error.std()
    events_df.loc[(events_df.event=='smp')&(events_df.carpenter_error<=avg_sac_carp_error_plus_two_std), 'event'] = 'sac'

    # remove saccades and smooth pursuits with amplitude below 1.5
    events_df = events_df.drop(events_df.loc[(events_df.amplitude<1.5)&(events_df.event=='sac')].index)
    events_df.reset_index(inplace=True)
    del events_df['index']
    events_df = events_df.drop(events_df.loc[(events_df.amplitude<1.5)&(events_df.event=='smp')].index)
    events_df.reset_index(inplace=True)
    del events_df['index']
    
    
    #
    # Algo 5.1: Merge around blinks
    #
    test_df = df.copy()
    test_events_df = events_df.copy()
    
    # Calculate statistical thresholds based on the data:
    # use two std for amplitude bc system noise in spatial data
    # use one std for carpenter bc error calculation
    avg_fix_amp_plus_two_std = test_events_df[test_events_df.event=='fix'].amplitude.mean()+2*test_events_df[test_events_df.event=='fix'].amplitude.std()
    avg_sac_carp_error_plus_two_std = test_events_df[test_events_df.event=='sac'].carpenter_error.mean()+2*test_events_df[test_events_df.event=='sac'].carpenter_error.std()
    avg_sac_calc_error_plus_two_std = test_events_df[test_events_df.event=='sac'].calculus_error.mean()+test_events_df[test_events_df.event=='sac'].calculus_error.std()

    to_drop = []
    for idx, row in test_events_df.copy().iterrows():
        if idx == 0 or idx == len(test_events_df)-1:
            continue

        prev_row = test_events_df.loc[idx-1]
        next_row = test_events_df.loc[idx+1]

        # if a blink or noise occurs between the same event
        if row.event == 'blink' and prev_row.event == next_row.event:

            # if the neighboring events are fixations whose centers vary less than a threshold,
            #  then merge the two neighboring events
            if prev_row.event == 'fix' and np.sqrt((next_row.center_x-prev_row.center_x)**2+(next_row.center_y-prev_row.center_y)**2) <= avg_fix_amp_plus_two_std:
                test_events_df = merge_events(test_df, test_events_df,idx-1,idx+1,'fix',interp_v, SAMPLE_RATE,blink=True)
                to_drop.extend([idx,idx+1])
                # Flag in df where the blink used to be
                test_df.loc[row.start_i:row.end_i, 'has_blink'] = True
            else:   
                amplitude = np.sqrt((next_row.x0-prev_row.xn)**2+(next_row.y0-prev_row.yn)**2)
                duration = next_row.start_s-prev_row.end_s
                carpenter_error = np.abs(duration*1000 - (21 + 2.2 * amplitude))/(21 + 2.2 * amplitude)
                # if the blink follows saccadic behavior according to carpenter, 
                # then it's likely a saccade with blink
                if carpenter_error <= avg_sac_carp_error_plus_two_std:
                    # if the neighboring events are also saccades, then merge into one saccade
                    if prev_row.event == 'sac':
                        test_events_df = merge_events(test_df, test_events_df,idx-1,idx+1, 'sac', interp_v, SAMPLE_RATE,blink=True)
                        to_drop.extend([idx,idx+1])
                    # otherwise reclassify the blink as a saccade
                    else:
                        test_df.loc[row.start_i:row.end_i, 'event'] = 'sac'
                        test_events_df.loc[idx, 'event'] = 'sac'
                        test_events_df.loc[idx, 'has_blink'] = True
                        # Flag in df where the blink used to be
                        test_df.loc[row.start_i:row.end_i, 'has_blink'] = True
                # otherwise merge the neigboring events, ignoring the blink
                else:
                    event = prev_row.event
                    test_events_df = merge_events(test_df, test_events_df,idx-1,idx+1,event,interp_v, SAMPLE_RATE,blink=True)
                    to_drop.extend([idx,idx+1])
                    # Flag in df where the blink used to be
                    test_df.loc[row.start_i:row.end_i, 'has_blink'] = True

        # Otherwise, if a blink breaks up separate events (neither of which is loss)
        elif row.event == 'blink' and prev_row.event!='loss' and next_row.event!='loss':
            amplitude = np.sqrt((next_row.x0-prev_row.xn)**2+(next_row.y0-prev_row.yn)**2)
            duration = next_row.start_s-prev_row.end_s
            carpenter_error = np.abs(duration*1000 - (21 + 2.2 * amplitude))/(21 + 2.2 * amplitude)
            # if the blink follows saccadic behavior according to carpenter, 
            # then it's likely a saccade with blink
            if carpenter_error <= avg_sac_carp_error_plus_two_std:
                test_df.loc[row.start_i:row.end_i, 'event'] = 'sac'
                test_events_df.loc[idx, 'event'] = 'sac'
                test_df.loc[row.start_i:row.end_i, 'has_blink'] = True
                test_events_df.loc[idx, 'has_blink'] = True
                # if one of the neighboring events is also a saccade, then merge them together
                if prev_row.event == 'sac':
                    test_events_df = merge_events(test_df, test_events_df,idx-1,idx,'sac',interp_v, SAMPLE_RATE,blink=True)
                    # Drop the event
                    to_drop.extend([idx])
                elif next_row.event == 'sac':
                    test_events_df = merge_events(test_df, test_events_df,idx,idx+1,'sac',interp_v, SAMPLE_RATE,blink=True)
                    # Drop the next event
                    to_drop.extend([idx+1])

            # otherwise, leave as blink
            else:
                test_df.loc[row.start_i:row.end_i, 'has_blink'] = True
                test_events_df.loc[idx, 'has_blink'] = True

    # drop events that are now merged
    test_events_df = test_events_df.drop(to_drop)
    test_events_df.reset_index(inplace=True)
    del test_events_df['index']

    # map the new events_df to df for plotting purposes
    test_df = map_to_stream(events_df, df)

    # adjust amplitude and related featurese
    test_events_df = adjust_amp_dur(test_events_df, test_df, interp_v)
    
    
    #
    # Algo 5.2: Find missing saccades between consecutive events (otherwise merge)
    #
    test_df_51 = test_df.copy()
    test_events_df_51 = test_events_df.copy()

    # find all consecutive events and add a saccade between them if there is a spatial jump
    # greater than threshold (otherwise merge)
    to_add, values = [], []
    test_events_df['drop'] = 0
    for idx, row in test_events_df.iterrows():
        if idx == 0:
            continue
        prev_row = test_events_df.loc[idx-1]
        if row.event == prev_row.event:
            x0, xn = test_df.loc[prev_row.end_i,'x_deg'], test_df.loc[row.start_i,'x_deg']
            y0, yn = test_df.loc[prev_row.end_i,'y_deg'], test_df.loc[row.start_i,'y_deg']
            t0, tn = test_df.loc[prev_row.end_i,'time'], test_df.loc[row.start_i,'time']
            if row.event == 'fix':
                # calculate distance between centers of consecutive events
                A = np.sqrt((row.center_x-prev_row.center_x)**2+(row.center_y-prev_row.center_y)**2)
            else:
                # calculate distance between last and first of samples of consecutive events
                A = np.sqrt((xn-x0)**2+(yn-y0)**2)
            # add a saccade there if it's greater than the fixation amplitude threshold
            if A > avg_fix_amp_plus_two_std:
                to_add.append(idx)
                D = tn - t0
                max_vel, avg_vel, std_vel = interpolate_velocity([t0, tn], interp_v, SAMPLE_RATE)
                avg_Dn = np.nanmean([prev_row.avg_Dn, row.avg_Dn])
                avg_iss = A/D
                carpenter_error = np.abs((D*1000 - (21 + 2.2 * A))/(21 + 2.2 * A))
                integral = interp_v.integral(t0, tn)
                calculus_error = np.abs((A-integral)/integral)
                values.append(['sac', #event
                               prev_row.end_i, #start_i
                               row.start_i, #end_i
                               t0,    #start_s
                               tn,    #end_s
                               D,   #duration
                               A,   #amplitude
                               np.nan,  #dispersion
                               np.nan, #center_x
                               np.nan, #center_y
                               x0, #x0
                               y0, #y0
                               xn, #xn
                               yn, #yn
                               np.nan, #std_x
                               np.nan, #std_y
                               np.nan, #cov_x
                               np.nan, #cov_y
                               np.nan, #cov_xy
                               np.nan, #max_Dn
                               np.nan, #avg_Dn
                               np.nan, #max_iss
                               avg_iss, #avg_iss
                               max_vel, #max_vel
                               avg_vel, #avg_vel
                               std_vel, #std_vel
                               np.nan, #P_fix
                               np.nan, #P_nonfix
                               np.nan, #max_P_nonfix
                               np.nan, #L_fix
                               np.nan, #L_nonfix
                               np.nan, #P_ff
                               np.nan, #P_smp
                               np.nan, #P_sac
                               np.nan, #P_blink
                               np.nan, #L_ff
                               np.nan, #L_smp
                               np.nan, #L_sac
                               np.nan, #L_blink
                               calculus_error, #calculus_error
                               carpenter_error, #carpenter_error
                               0, #has_blink
                               0]) #drop

            # otherwise merge the events
            else:
                # if the previous event has already been merged, then merge current event with the
                # first event in the consecutive sequence
                i = idx
                while i > 0 and test_events_df.loc[i-1, 'drop'] == 1 and row.event == test_events_df.loc[i-1, 'event']:
                    i -= 1
                event = test_events_df.loc[idx, 'event']
                test_events_df = merge_events(test_df, test_events_df,i-1,idx,event,interp_v, SAMPLE_RATE)
                test_events_df.loc[idx, 'drop'] = 1

    # add the additional saccades to the data, if any
    if len(to_add) > 0:
        test_events_df = pd.DataFrame(np.insert(test_events_df.values, to_add, values, axis=0))
        test_events_df.columns = events_df.columns.tolist()+['drop']

    # remove any rows that have been flagged as drop
    to_drop = test_events_df[test_events_df['drop']==1].index.tolist()

    if len(to_drop) > 0:
        # drop events that are now merged
        test_events_df = test_events_df.drop(to_drop)
        test_events_df.reset_index(inplace=True)
        del test_events_df['index']

        # map the new events_df to df for plotting purposes
        test_df = map_to_stream(events_df, df)

        # adjust endpoints, amplitude, duration
        test_events_df = adjust_amp_dur(test_events_df, df, interp_v)
    
    
    #
    # Step 5.3: Add final physiological checks
    #
    test_df_52 = test_df.copy()
    test_events_df_52 = test_events_df.copy()
    
    # remove saccades and smooth pursuits with amplitude below 1.5
    test_events_df = test_events_df.drop(test_events_df.loc[(test_events_df.amplitude<1.5)&(test_events_df.event=='sac')].index)
    test_events_df.reset_index(inplace=True)
    del test_events_df['index']
    test_events_df = test_events_df.drop(test_events_df.loc[(test_events_df.amplitude<1.5)&(test_events_df.event=='smp')].index)
    test_events_df.reset_index(inplace=True)
    del test_events_df['index']
    
    # remove fixations with duration less than 100 ms
    test_events_df = test_events_df.drop(test_events_df.loc[(test_events_df.duration<0.1)&(test_events_df.event=='fix')].index)
    test_events_df.reset_index(inplace=True)
    del test_events_df['index']
    
    # remove events with calculus error greater than 1.5
    test_events_df = test_events_df.drop(test_events_df.loc[test_events_df.calculus_error>1.5].index)
    test_events_df.reset_index(inplace=True)
    del test_events_df['index']
    
    # if saccades are longer than the maximum possible duration of a saccade (0.613sec) - this is a check
#     # (given that the max span of human vision is 180deg horizontal and 200deg vertical and max vel is 1000deg/s)
#     max_sac_dur = 0.6
#     test_events_df.loc[test_events_df[(test_events_df.event=='sac') & (test_events_df.duration > max_sac_dur)].index.tolist(),'event']='noise'

        
    #
    # Step 6: Append to output event sequence
    #
    events_df['subID'] = subID
    events_df['task'] = task
    events_df['VL'] = vision_loss[subID]
    events_df['HMD'] = DEVICE
    events_df['eye'] = eye
    events_df['freq'] = sample_rates[DEVICE]
    # Want one combined csv of sequences, all events_df labelled by device, subID, VL, and task
    sequences = pd.DataFrame({'HMD':events_df.HMD,
                            'rate':events_df.freq,
                            'eye':events_df.eye,
                            'task':events_df.task,
                            'subID':events_df.subID,
                            'VL':events_df.VL,
                            'event':events_df.event,
                            'start_i':events_df.start_i,
                            'end_i':events_df.end_i,
                            'start_s':events_df.start_s,
                            'end_s':events_df.end_s,
                            'center_x':events_df.center_x,
                            'center_y':events_df.center_y,
                            'x0':events_df.x0,
                            'y0':events_df.y0,
                            'xn':events_df.xn,
                            'yn':events_df.yn,
                            'cov_x':events_df.cov_x,
                            'cov_y':events_df.cov_y,
                            'cov_xy':events_df.cov_xy,
                            'amplitude':events_df.amplitude,
                            'dispersion':events_df.dispersion,
                            'duration':events_df.duration,
                            'max_vel':events_df.max_vel,
                            'avg_vel':events_df.avg_vel,
                            'std_vel':events_df.std_vel,
                            'max_Dn':events_df.max_Dn,
                            'avg_Dn':events_df.avg_Dn,
                            'max_iss':events_df.max_iss,
                            'avg_iss':events_df.avg_iss,
                            'calculus_error':events_df.calculus_error,
                            'carpenter_error':events_df.carpenter_error,
                            'P_nonfix':events_df.P_nonfix,
                            'P_fix':events_df.P_fix,
                            'P_ff':events_df.P_ff,
                            'P_smp':events_df.P_smp,
                            'P_sac':events_df.P_sac,
                            'P_blink':events_df.P_blink})
    test_seq = pd.DataFrame({'HMD':DEVICE,
                            'rate':sample_rates[DEVICE],
                            'eye':eye,
                            'task':task,
                            'subID':subID,
                            'VL':vision_loss[subID],
                            'event':test_events_df.event,
                            'start_i':test_events_df.start_i,
                            'end_i':test_events_df.end_i,
                            'start_s':test_events_df.start_s,
                            'end_s':test_events_df.end_s,
                            'center_x':test_events_df.center_x,
                            'center_y':test_events_df.center_y,
                            'x0':test_events_df.x0,
                            'y0':test_events_df.y0,
                            'xn':test_events_df.xn,
                            'yn':test_events_df.yn,
                            'cov_x':test_events_df.cov_x,
                            'cov_y':test_events_df.cov_y,
                            'cov_xy':test_events_df.cov_xy,
                            'amplitude':test_events_df.amplitude,
                            'dispersion':test_events_df.dispersion,
                            'duration':test_events_df.duration,
                            'max_vel':test_events_df.max_vel,
                            'avg_vel':test_events_df.avg_vel,
                            'std_vel':test_events_df.std_vel,
                            'max_Dn':test_events_df.max_Dn,
                            'avg_Dn':test_events_df.avg_Dn,
                            'max_iss':test_events_df.max_iss,
                            'avg_iss':test_events_df.avg_iss,
                            'calculus_error':test_events_df.calculus_error,
                            'carpenter_error':test_events_df.carpenter_error,
                            'P_nonfix':test_events_df.P_nonfix,
                            'P_fix':test_events_df.P_fix,
                            'P_ff':test_events_df.P_ff,
                            'P_smp':test_events_df.P_smp,
                            'P_sac':test_events_df.P_sac,
                            'P_blink':test_events_df.P_blink,
                            'has_blink':test_events_df.has_blink})
    test_seq_51 = pd.DataFrame({'HMD':DEVICE,
                            'rate':sample_rates[DEVICE],
                            'eye':eye,
                            'task':task,
                            'subID':subID,
                            'VL':vision_loss[subID],
                            'event':test_events_df_51.event,
                            'start_i':test_events_df_51.start_i,
                            'end_i':test_events_df_51.end_i,
                            'start_s':test_events_df_51.start_s,
                            'end_s':test_events_df_51.end_s,
                            'center_x':test_events_df_51.center_x,
                            'center_y':test_events_df_51.center_y,
                            'x0':test_events_df_51.x0,
                            'y0':test_events_df_51.y0,
                            'xn':test_events_df_51.xn,
                            'yn':test_events_df_51.yn,
                            'cov_x':test_events_df_51.cov_x,
                            'cov_y':test_events_df_51.cov_y,
                            'cov_xy':test_events_df_51.cov_xy,
                            'amplitude':test_events_df_51.amplitude,
                            'dispersion':test_events_df_51.dispersion,
                            'duration':test_events_df_51.duration,
                            'max_vel':test_events_df_51.max_vel,
                            'avg_vel':test_events_df_51.avg_vel,
                            'std_vel':test_events_df_51.std_vel,
                            'max_Dn':test_events_df_51.max_Dn,
                            'avg_Dn':test_events_df_51.avg_Dn,
                            'max_iss':test_events_df_51.max_iss,
                            'avg_iss':test_events_df_51.avg_iss,
                            'calculus_error':test_events_df_51.calculus_error,
                            'carpenter_error':test_events_df_51.carpenter_error,
                            'P_nonfix':test_events_df_51.P_nonfix,
                            'P_fix':test_events_df_51.P_fix,
                            'P_ff':test_events_df_51.P_ff,
                            'P_smp':test_events_df_51.P_smp,
                            'P_sac':test_events_df_51.P_sac,
                            'P_blink':test_events_df_51.P_blink,
                            'has_blink':test_events_df_51.has_blink})
    test_seq_52 = pd.DataFrame({'HMD':DEVICE,
                            'rate':sample_rates[DEVICE],
                            'eye':eye,
                            'task':task,
                            'subID':subID,
                            'VL':vision_loss[subID],
                            'event':test_events_df_52.event,
                            'start_i':test_events_df_52.start_i,
                            'end_i':test_events_df_52.end_i,
                            'start_s':test_events_df_52.start_s,
                            'end_s':test_events_df_52.end_s,
                            'center_x':test_events_df_52.center_x,
                            'center_y':test_events_df_52.center_y,
                            'x0':test_events_df_52.x0,
                            'y0':test_events_df_52.y0,
                            'xn':test_events_df_52.xn,
                            'yn':test_events_df_52.yn,
                            'cov_x':test_events_df_52.cov_x,
                            'cov_y':test_events_df_52.cov_y,
                            'cov_xy':test_events_df_52.cov_xy,
                            'amplitude':test_events_df_52.amplitude,
                            'dispersion':test_events_df_52.dispersion,
                            'duration':test_events_df_52.duration,
                            'max_vel':test_events_df_52.max_vel,
                            'avg_vel':test_events_df_52.avg_vel,
                            'std_vel':test_events_df_52.std_vel,
                            'max_Dn':test_events_df_52.max_Dn,
                            'avg_Dn':test_events_df_52.avg_Dn,
                            'max_iss':test_events_df_52.max_iss,
                            'avg_iss':test_events_df_52.avg_iss,
                            'calculus_error':test_events_df_52.calculus_error,
                            'carpenter_error':test_events_df_52.carpenter_error,
                            'P_nonfix':test_events_df_52.P_nonfix,
                            'P_fix':test_events_df_52.P_fix,
                            'P_ff':test_events_df_52.P_ff,
                            'P_smp':test_events_df_52.P_smp,
                            'P_sac':test_events_df_52.P_sac,
                            'P_blink':test_events_df_52.P_blink,
                            'has_blink':test_events_df_52.has_blink})
    
    #
    # save results
    #
    sequences.to_csv(f'{out_path}50/{ID}_50.csv')
    test_seq_51.to_csv(f'{out_path}51/{ID}_51.csv')
    test_seq_52.to_csv(f'{out_path}52/{ID}_52.csv')
    test_seq.to_csv(f'{out_path}{ID}.csv')
    print(f'[{NUM}] {ID} success!')
    NUM += 1

varjo_cereal_right_P19 processing...
[1] varjo_cereal_right_P19 success!
varjo_cereal_left_P19 processing...
[2] varjo_cereal_left_P19 success!
varjo_sandwich_left_P16 processing...
[3] varjo_sandwich_left_P16 success!
varjo_sandwich_left_P24 processing...
[4] varjo_sandwich_left_P24 success!
varjo_cereal_left_P24 processing...
[5] varjo_cereal_left_P24 success!
varjo_cereal_left_P6 processing...
[6] varjo_cereal_left_P6 success!
varjo_cereal_left_P5 processing...
[7] varjo_cereal_left_P5 success!
varjo_cereal_right_P24 processing...
[8] varjo_cereal_right_P24 success!
varjo_cereal_right_P21 processing...
[9] varjo_cereal_right_P21 success!
varjo_sandwich_right_P16 processing...
[10] varjo_sandwich_right_P16 success!
varjo_cereal_left_P9 processing...
[11] varjo_cereal_left_P9 success!
varjo_cereal_left_P14 processing...
[12] varjo_cereal_left_P14 success!
varjo_cereal_left_P16 processing...
[13] varjo_cereal_left_P16 success!
varjo_cereal_right_P5 processing...
[14] varjo_cereal_right

In [8]:
# merge results - 5.0
in_path = '/data/Isabella/thesis_spring2022/event_detect_out_final/50/'
i = 0
for fname in os.listdir(in_path):
    if 'all_sequences' in fname or 'PI' in fname:
        continue
    if i == 0:
        seq = pd.read_csv(in_path+fname)
        i += 1
    else:
        seq = pd.concat([seq, pd.read_csv(in_path+fname)])
seq.to_csv(in_path+'all_sequences_50_varjo.csv')

In [9]:
# merge results - 5.1
in_path = '/data/Isabella/thesis_spring2022/event_detect_out_final/51/'
i = 0
for fname in os.listdir(in_path):
    if 'all_sequences' in fname or 'PI' in fname:
        continue
    if i == 0:
        seq = pd.read_csv(in_path+fname)
        i += 1
    else:
        seq = pd.concat([seq, pd.read_csv(in_path+fname)])
seq.to_csv(in_path+'all_sequences_51_varjo.csv')

In [10]:
# merge results - 5.2
in_path = '/data/Isabella/thesis_spring2022/event_detect_out_final/52/'
i = 0
for fname in os.listdir(in_path):
    if 'all_sequences' in fname or 'PI' in fname:
        continue
    if i == 0:
        seq = pd.read_csv(in_path+fname)
        i += 1
    else:
        seq = pd.concat([seq, pd.read_csv(in_path+fname)])
seq.to_csv(in_path+'all_sequences_52_varjo.csv')

In [12]:
# merge results - final
in_path = '/data/Isabella/thesis_spring2022/event_detect_out_final/'
i = 0
for fname in os.listdir(in_path):
    if fname == 'archive' or 'all_sequences' in fname or 'PI' in fname:
        continue
    if '50' not in fname and '51' not in fname and '52' not in fname:
        if i == 0:
            seq = pd.read_csv(in_path+fname)
            i += 1
        else:
            seq = pd.concat([seq, pd.read_csv(in_path+fname)])
seq.to_csv(in_path+'all_sequences_varjo.csv')